In [1]:
#########################################################################################################################
# COLLECT DATA FOR NEXT GAME DAY #

# Script 2 of 4
# This script collects data for the next game day.
# Ensure `_1. 03012025_get_data_previous_game_day and parse to statistics.ipynb` is executed before running this script.
#########################################################################################################################


In [2]:
# Define the current season
current_season = 2025

In [3]:
from bs4 import BeautifulSoup
from datetime import datetime
import os
import requests
import calendar
import pandas as pd
import shutil
from datetime import datetime, timedelta

In [4]:
# Calculate today's date and month

today = datetime.now() - timedelta(days=0)
today_str = today.strftime("%a, %b ") + str(int(today.strftime("%d"))) + today.strftime(", %Y")
today_date = datetime.strptime(today_str, "%a, %b %d, %Y")

today_str_format = today_date.strftime("%Y-%m-%d")

print(f"Today's date: {today_str}")

month_for_coming_games = today.month
month_name_for_coming_games = calendar.month_name[month_for_coming_games].lower()
print(month_name_for_coming_games)

Today's date: Fri, Jan 17, 2025
january


In [5]:
# Directories

DATA_DIR = os.path.join("D:\\", "1. Python", "1. NBA Script","2025", "Gathering_Data", "data")
STANDINGS_DIR = os.path.join(DATA_DIR, f"{current_season}_standings")

# Define the target folder path
target_folder = r'D:\1. Python\1. NBA Script\2025\Gathering_Data\Next_Game'
# Source and destination directories
dst_dir = r'D:\_Laufwerk C\11. Sorare\NBA\2025\Gathering_Data\Next_Game'

file_name = f"NBA_{current_season}_games-{month_name_for_coming_games}.html"
file_path = os.path.join(STANDINGS_DIR, file_name)

print(f"Checking file: {file_path}")

Checking file: D:\1. Python\1. NBA Script\2025\Gathering_Data\data\2025_standings\NBA_2025_games-january.html


In [6]:
# Function

def scrape_season_for_month(season, month, month_name, standings_dir):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    print(url)
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all("a", href=True)
        month_link = None
        for link in links:
            if f"NBA_{season}_games-{month_name}" in link['href']:
                month_link = "https://www.basketball-reference.com" + link['href']
                break
        if month_link:
            month_response = requests.get(month_link)
            month_response.raise_for_status()
            with open(os.path.join(standings_dir, f"NBA_{season}_games-{month_name}.html"), "w", encoding='utf-8') as f:
                f.write(month_response.text)
            print(f"Data for {month_name.title()} saved.")
        else:
            print(f"No link found for {month_name.title()} {season}.")
    except requests.RequestException as e:
        print(f"An error occurred while fetching data: {e}")

###############################################################################################################################

#########################################################################################
#from datetime import datetime
#from bs4 import BeautifulSoup
#import os

def find_games_for_next_day(today_date, file_paths):
    next_game_info = []
    next_game_date = None  # Placeholder to track the date of the first game found

    for file_path in file_paths:
        if not os.path.exists(file_path):
            continue

        try:
            with open(file_path, "r", encoding='utf-8') as file:
                soup = BeautifulSoup(file.read(), 'html.parser')

            table = soup.find("table", {"id": "schedule"})

            if table:
                rows = table.find_all("tr")
                for row in rows[1:]:  # Skip the header row
                    game_date_tag = row.find("th", {"data-stat": "date_game"})
                    if game_date_tag:
                        game_date_str = game_date_tag.text.strip()
                        try:
                            game_date = datetime.strptime(game_date_str, "%a, %b %d, %Y")
                        except ValueError:
                            continue

                        # If it's the first game on or after today_date, set next_game_date
                        if game_date >= today_date and next_game_date is None:
                            next_game_date = game_date

                        # Collect games for the determined next game date
                        if game_date == next_game_date:
                            cols = row.find_all("td")
                            if len(cols) >= 4:
                                visitor_team = cols[1].text.strip()
                                home_team = cols[3].text.strip()
                                next_game_info.append({
                                    'date': game_date,
                                    'home_team': home_team,
                                    'visitor_team': visitor_team
                                })
                        # Stop once we've collected all games for the next available game day
                        elif next_game_date is not None and game_date > next_game_date:
                            return next_game_info

        except Exception as e:
            print(f"An error occurred while processing the file: {e}")

    return next_game_info


###############################################################################################################################

print(file_path)

# Scrape data for the current month if not present
if not os.path.exists(file_path):
    print(f"File does not exist: {file_path}")
    scrape_season_for_month(current_season, month_for_coming_games, month_name_for_coming_games, STANDINGS_DIR)

next_game_info = find_games_for_next_day(today_date, [file_path])
#next_game_info = find_games_for_today(today_date, [file_path])
#print(next_game_info)

###############################################################################################################################

# Prepare to look ahead one month at a time
next_month = month_for_coming_games % 12 + 1  # Handle month transition properly
months_checked = 0
max_months_to_check = 12  # Check up to 12 months ahead
#next_game_info = None

while not next_game_info and months_checked < max_months_to_check:
    month_name_for_coming_games = calendar.month_name[next_month].lower()
    file_name = f"NBA_{current_season}_games-{month_name_for_coming_games}.html"
    file_path = os.path.join(STANDINGS_DIR, file_name)
    print(file_path,'AAAA')
    
    if not os.path.exists(file_path):
        scrape_season_for_month(current_season, next_month, month_name_for_coming_games, STANDINGS_DIR)

    next_game_info = find_next_game(today_date, [file_path])
    next_month = (next_month % 12) + 1  # Move to the next month
    months_checked += 1

# Display the next game
# Map team names to their codes and create a DataFrame
team_codes = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BRK',
    'Charlotte Hornets': 'CHO',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}



D:\1. Python\1. NBA Script\2025\Gathering_Data\data\2025_standings\NBA_2025_games-january.html


In [7]:
games = []

# Display all the games for today
if next_game_info:
    for game in next_game_info:  # Loop through the list of games
        home_team_name = team_codes.get(game['home_team'], game['home_team'])  # Use 'game' instead of 'next_game_info'
        visitor_team_name = team_codes.get(game['visitor_team'], game['visitor_team'])
        games.append((home_team_name, visitor_team_name, game['date'].strftime("%Y-%m-%d")))
    
        print(f"The game is on {game['date'].strftime('%a, %b %d, %Y')}.")
        print(f"Teams: {game['visitor_team']} vs {game['home_team']}")
else:
    print("No games found for today.")

# Create DataFrame
games_df = pd.DataFrame(games, columns=['home_team', 'away_team', 'game_date'])

print(games_df)


The game is on Fri, Jan 17, 2025.
Teams: Orlando Magic vs Boston Celtics
The game is on Fri, Jan 17, 2025.
Teams: Minnesota Timberwolves vs New York Knicks
The game is on Fri, Jan 17, 2025.
Teams: Charlotte Hornets vs Chicago Bulls
The game is on Fri, Jan 17, 2025.
Teams: Denver Nuggets vs Miami Heat
The game is on Fri, Jan 17, 2025.
Teams: Toronto Raptors vs Milwaukee Bucks
The game is on Fri, Jan 17, 2025.
Teams: Utah Jazz vs New Orleans Pelicans
The game is on Fri, Jan 17, 2025.
Teams: Oklahoma City Thunder vs Dallas Mavericks
The game is on Fri, Jan 17, 2025.
Teams: Memphis Grizzlies vs San Antonio Spurs
The game is on Fri, Jan 17, 2025.
Teams: Brooklyn Nets vs Los Angeles Lakers
  home_team away_team   game_date
0       BOS       ORL  2025-01-17
1       NYK       MIN  2025-01-17
2       CHI       CHO  2025-01-17
3       MIA       DEN  2025-01-17
4       MIL       TOR  2025-01-17
5       NOP       UTA  2025-01-17
6       DAL       OKC  2025-01-17
7       SAS       MEM  2025-01-17
8

In [8]:
# Create the folder if it doesn't exist
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# Save the games_df DataFrame to a CSV file in the target folder
games_df.to_csv(os.path.join(target_folder, 'games_df_' + today_str_format + '.csv'))#, index=False)

print(f"games_df has been saved in the folder: {target_folder}")


games_df has been saved in the folder: D:\1. Python\1. NBA Script\2025\Gathering_Data\Next_Game


In [9]:
#import shutil
#import os

# Check if the source folder exists
if os.path.exists(target_folder) and os.path.isdir(target_folder):
    # Create the destination folder if it does not exist
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    # Get a list of all files in the source directory
    files_to_copy = [file_name for file_name in os.listdir(target_folder) if not file_name.startswith('.ipynb_checkpoints')]
    
    # Flag to check if any new files were copied
    new_files_copied = False

    # Copy each file from source to destination if it does not already exist
    for file_name in files_to_copy:
        target_folder_file_path = os.path.join(target_folder, file_name)
        dst_file_path = os.path.join(dst_dir, file_name)

        # Copy the file if it does not exist in the destination folder
        if not os.path.exists(dst_file_path):
            shutil.copy2(target_folder_file_path, dst_file_path)
            print(f'File "{file_name}" copied successfully.')
            new_files_copied = True

    # If no new files were copied, display a message
    if not new_files_copied:
        print("No new files to copy.")

else:
    print(f'Source folder "{target_folder}" does not exist or is not a directory.')


File "best_home_games_2025-01-16.csv" copied successfully.
File "games_df_2025-01-17.csv" copied successfully.
